In [5]:
%spark

// Import necessary libraries
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.IndexToString
import org.apache.spark.sql.SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.IndexToString
import org.apache.spark.sql.SparkSession


In [11]:
%spark
spark.catalog.setCurrentDatabase("census_income_db")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
spark.catalog.listTables().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------+------------------+-----------+---------+-----------+
|            name|      catalog|         namespace|description|tableType|isTemporary|
+----------------+-------------+------------------+-----------+---------+-----------+
|adult_data_clean|spark_catalog|[census_income_db]|       NULL| EXTERNAL|      false|
|   census_income|spark_catalog|[census_income_db]|       NULL| EXTERNAL|      false|
+----------------+-------------+------------------+-----------+---------+-----------+



In [13]:
val df = spark.sql("select * from adult_data_clean where age is not null ")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [age: bigint, workclass: string ... 5 more fields]


In [14]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- age: long (nullable = true)
 |-- workclass: string (nullable = true)
 |-- education: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- country: string (nullable = true)
 |-- income_cat: string (nullable = true)



In [16]:
df.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------------+------------+--------------------+--------------------+----------------+----------+
|age|          workclass|   education|        relationship|          occupation|         country|income_cat|
+---+-------------------+------------+--------------------+--------------------+----------------+----------+
| 39|       " State-gov"|" Bachelors"|    " Never-married"|     " Adm-clerical"|" United-States"|  " <=50K"|
| 50|" Self-emp-not-inc"|" Bachelors"|" Married-civ-spo...|  " Exec-managerial"|" United-States"|  " <=50K"|
| 38|         " Private"|  " HS-grad"|         " Divorced"|" Handlers-cleaners"|" United-States"|  " <=50K"|
| 53|         " Private"|     " 11th"|" Married-civ-spo...|" Handlers-cleaners"|" United-States"|  " <=50K"|
| 28|         " Private"|" Bachelors"|" Married-civ-spo...|   " Prof-specialty"|         " Cuba"|  " <=50K"|
| 37|         " Private"|  " Masters"|" Married-civ-spo...|  " Exec-managerial"|" United-States"|  " <=50K"|
| 49|         " Pri

In [23]:
val trainingSize = 0.8
val testSize = 0.2

val workclass_indexer = new StringIndexer().setInputCol("workclass").setOutputCol("workclass-index").fit(df)
val edu_indexer = new StringIndexer().setInputCol("education").setOutputCol("edu-index").fit(df)
val relationship_indexer = new StringIndexer().setInputCol("relationship").setOutputCol("reln-index").fit(df)
val occupation_indexer = new StringIndexer().setInputCol("occupation").setOutputCol("occu-index").fit(df)
val income_indexer = new StringIndexer().setInputCol("income_cat").setOutputCol("category-index").fit(df)
val assembler = new VectorAssembler().setInputCols(Array("age", "workclass-index", "edu-index", "reln-index", "occu-index")).setOutputCol("features")


// val workclass_indexer = new StringIndexer().setInputCol("workclass").setOutputCol("workclass-index").fit(df)
// val edu_indexer = new StringIndexer().setInputCol("education").setOutputCol("edu-index").fit(df)
// val relationship_indexer = new StringIndexer().setInputCol("relationship").setOutputCol("reln-index").fit(df)
// val occupation_indexer = new StringIndexer().setInputCol("occupation").setOutputCol("occu-index").fit(df)
// val income_indexer = new StringIndexer().setInputCol("income_cat").setOutputCol("category-index").fit(df)
// val assembler = new VectorAssembler().setInputCols(Array("age", "workclass-index", "edu-index", "reln-index", "occu")).setOutputCol("features")

val Array(trainingData, testData) = df.randomSplit(Array(trainingSize, testSize))
println("trainingData count:" + trainingData.count)
println("testData count:" + testData.count)

// val Array(trainingData, testData) = df.randomSplit(Array(trainingSize, testSize))
// println("trainingData count:" + trainingData.count)
// println("testSize count:" + testData.count)

val dt = new DecisionTreeClassifier().setLabelCol("category-index").setFeaturesCol("features")
val labelConverter = new IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(income_indexer.labels)
val pipeline = new Pipeline().setStages(Array(workclass_indexer, edu_indexer, relationship_indexer, occupation_indexer, assembler, income_indexer, dt, labelConverter))

// val dt = new DecisionTreeClassifier().setLabelCol("category-index").setFeaturesCol("features")
// val labelConverter = new IndexToString().setInputCol("prediction").setOutputCol("predictedLabels").setLabels(income_indexer.labels)
// val pipeline = new Pipeline().setStages(Array(workclass_indexer, edu_indexer, relationship_indexer, assembler, income_indexer, dt, labelConverter))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingSize: Double = 0.8
testSize: Double = 0.2
workclass_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_17512ee43e42, handleInvalid=error
edu_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_8190ce8fc158, handleInvalid=error
relationship_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_82c2a7d754a2, handleInvalid=error
occupation_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_4352918da52e, handleInvalid=error
income_indexer: org.apache.spark.ml.feature.StringIndexerModel = StringIndexerModel: uid=strIdx_7a46720bc5ea, handleInvalid=error
assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_a9549acd1fc3, handleInvalid=error, numInputCols=5
trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [age: bigint, workclass: string ... 5 more fields]
testData: org.apache.spark.

In [24]:
val model = pipeline.fit(trainingData)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model: org.apache.spark.ml.PipelineModel = pipeline_63e5e35103ca


In [25]:
val preds = model.transform(testData)
val preds_records = preds.select($"age", $"workclass", $"education", $"relationship", $"probability", $"predictedLabel")
preds_records.show(20)
println("count:" + preds_records.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

preds: org.apache.spark.sql.DataFrame = [age: bigint, workclass: string ... 15 more fields]
preds_records: org.apache.spark.sql.DataFrame = [age: bigint, workclass: string ... 4 more fields]
+---+-------------------+---------+----------------+--------------------+--------------+
|age|          workclass|education|    relationship|         probability|predictedLabel|
+---+-------------------+---------+----------------+--------------------+--------------+
| 17|               " ?"|  " 10th"|" Never-married"|[0.96743482516238...|      " <=50K"|
| 17|               " ?"|  " 11th"|" Never-married"|[0.96743482516238...|      " <=50K"|
| 17|               " ?"|  " 11th"|" Never-married"|[0.96743482516238...|      " <=50K"|
| 17|       " Local-gov"|   " 9th"|" Never-married"|[0.96743482516238...|      " <=50K"|
| 17|         " Private"|  " 10th"|" Never-married"|[0.96743482516238...|      " <=50K"|
| 17|         " Private"|  " 10th"|" Never-married"|[0.96743482516238...|      " <=50K"|
| 17|   

In [26]:
val pred_50Kover = preds_records.filter($"predictedLabel".contains(">50K"))
pred_50Kover.show(10)
println("count:" + pred_50Kover.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pred_50Kover: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [age: bigint, workclass: string ... 4 more fields]
+---+------------+--------------+--------------------+--------------------+--------------+
|age|   workclass|     education|        relationship|         probability|predictedLabel|
+---+------------+--------------+--------------------+--------------------+--------------+
| 26|  " Private"|  " Bachelors"|" Married-civ-spo...|[0.45370370370370...|       " >50K"|
| 26|  " Private"|  " Bachelors"|" Married-civ-spo...|[0.45370370370370...|       " >50K"|
| 27|  " Private"|  " Bachelors"|" Married-civ-spo...|[0.45370370370370...|       " >50K"|
| 28|" Local-gov"|    " Masters"|" Married-civ-spo...|[0.45370370370370...|       " >50K"|
| 28|  " Private"|  " Bachelors"|" Married-civ-spo...|[0.45370370370370...|       " >50K"|
| 28|  " Private"|  " Bachelors"|" Married-civ-spo...|[0.45370370370370...|       " >50K"|
| 28|  " Private"|  " Bachelors"|" Married-civ-spo...|[0.45

In [27]:
import org.apache.spark.ml.classification.DecisionTreeClassificationModel
val treeModel = model.stages(6).asInstanceOf[DecisionTreeClassificationModel]
println("Learned classification tree model:\n" + treeModel.toDebugString)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.ml.classification.DecisionTreeClassificationModel
treeModel: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel: uid=dtc_7f08933daa19, depth=5, numNodes=21, numClasses=2, numFeatures=5
Learned classification tree model:
DecisionTreeClassificationModel: uid=dtc_7f08933daa19, depth=5, numNodes=21, numClasses=2, numFeatures=5
  If (feature 3 in {1.0,2.0,3.0,4.0,5.0,6.0})
   If (feature 4 in {1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0})
    Predict: 0.0
   Else (feature 4 not in {1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0})
    If (feature 0 <= 27.5)
     Predict: 0.0
    Else (feature 0 > 27.5)
     If (feature 2 in {0.0,1.0,4.0,5.0,6.0,7.0,8.0,10.0,11.0,14.0})
      Predict: 0.0
     Else (feature 2 not in {0.0,1.0,4.0,5.0,6.0,7.0,8.0,10.0,11.0,14.0})
      If (feature 2 in {9.0,12.0})
       Predict: 1.0
      Else (feature 2 not in {9.0,12.0})
       Predict: 0.0
  Else (feature 3 not 